<a href="https://colab.research.google.com/github/Roubish/Sentiment-Classifier/blob/main/Sentiment_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How to Build a Sentiment Classifier using Python and IMDB Reviews

### About the Project
In this project, we will be primarily working on the IMDB reviews given by the viewers about various movies.

We will be getting the dataset from the tensorflow_datasets. Then we will develop an understanding of how to preprocess the data, convert the English words to numerical representations, and prepare it to be fed as input for our deep learning model with GRUs.

## Importing the Modules
Let us begin by importing the necessary modules, and setting the random seed so as to get reproducible results.

In [3]:
# Import numpy as np.
import numpy as np
# Import tensorflow as tf.
import tensorflow as tf
# Import keras from tensorflow.
from tensorflow import keras
#Set tensorflow seed:
tf.random.set_seed(42)

### Loading the Dataset
First, we will load the original IMDb reviews, as text (byte strings), using TensorFlow Datasets.

In [4]:
# import tensorflow_datasets as tfds.

import tensorflow_datasets as tfds

In [5]:
# Use load function of tfds.
# Pass "imdb_reviews" as argument.
# Set as_supervised=True and with_info=True.
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8QGFKE/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8QGFKE/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8QGFKE/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
# Use dataset.keys() to see the keys of datasets.

print(datasets.keys())

dict_keys(['test', 'train', 'unsupervised'])


In [7]:
# Let us see the size of the train data and test data.

train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

print(train_size , test_size)

25000 25000


### Exploring the Data
Let us see how the data looks like.
##### Note:

datasets["train"] contains the train data. Similarly, datasets["test"] contains the test data.

datasets["train"].batch(2) batches 2 data samples at a time.

datasets["train"].batch(2).take(1) allows to take 1 batch at a time.

Each batch is of type Eager Temsor. We could convert it to numpy array using X_batch.numpy().



In [8]:
# We shall traverse through the batches and show the review(first 200 characters) and label of the first batch data samples:

for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



### Defining the preprocess function
Now we will create this preprocessing function where we will:

Truncate the reviews, keeping only the first 300 characters of each since you can generally tell whether a review is positive or not in the first sentence or two.

Then we use regular expressions to replace <br/> tags with spaces and to replace any characters other than letters and quotes with spaces.

Finally, the preprocess() function splits the reviews by the spaces, which returns a ragged tensor, and it converts this ragged tensor to a dense tensor, padding all reviews with the padding token <pad> so that they all have the same length.

##### Note:

tf.strings - Operations for working with string Tensors.

tf.strings.substr(X_batch, 0, 300) - For each string in the input Tensor X_batch, it creates a substring starting at index pos(here 0) with a total length of len(here 300). So basically, it returns substrings from Tensor of strings.

tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ") - Replaces elements of X_batch matching regex pattern <br\s*/?> with rewrite .

tf.strings.split(X_batch) - Split elements of input X_batch into a RaggedTensor.

X_batch.to_tensor(default_value=b"<pad>") - Converts the RaggedTensor into a tf.Tensor. default_value is the value to set for indices not specified in X_batch. Empty values are assigned default_value(here <pad>).

In [9]:
# Use the following code to preprocess the data as described above:

def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [11]:
# Let us now call the preprocess() function on X_batch, y_batch to see how the data after preprocessing looks like:

preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

### Constructing the Vocabulary
Next, we will construct the vocabulary. This requires going through the whole training set once, applying our preprocess() function, and using a Counter() to count the number of occurrences of each word.

### Note:

- Counter().update() : We can add values to the Counter by using update() method.

- map(myfunc) of the tensorflow datasets maps the function(or applies the function) myfunc across all the samples of the given dataset

In [12]:
# Import Counter from collections.

from collections import Counter

In [13]:
# Get code Counter() object vocabulary.
vocabulary = Counter()

In [14]:
# For each review in every batch of the train data, let us make a vocabulary dictionary containing the words and their counts correspondingly:

for X_batch, y_batch in datasets["train"].batch(2).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [15]:
# Let's look at the 5 most common words
vocabulary.most_common()[:5]

[(b'<pad>', 63155),
 (b'the', 61137),
 (b'a', 38564),
 (b'of', 33983),
 (b'and', 33431)]

In [16]:
# Let us find the length of thre vocabulary using len functiuon
len(vocabulary)

53893

## Truncating the Vocabulary
There are more than 50,000 words in the vocabulary. So let us truncate it to have only 10,000 most common words.

In [17]:
# Set vocab_size to 10000
vocab_size = 10000

In [18]:
# Extract the top 10,000 most frequently occurring words from vocabulary and store these words in truncated_vocabulary list(let us use the list comprehension method to do so).

truncated_vocabulary = [ word for word, count in vocabulary.most_common()[:vocab_size]]

## Creating a lookup table
Computer can only process numbers but not words. Thus we need to convert the words in truncated_vocabulary into numbers.

So we now need to add a preprocessing step to replace each word with its ID (i.e., its index in the truncated_vocabulary). We will create a lookup table for this, using 1,000 out-of-vocabulary (oov) buckets.

We shall create the lookup table such that the most frequently occurring words have lower indices than less frequently occurring words.

##### Note:

- tf.lookup.KeyValueTensorInitializer : Table initializer given keys and values tensors. More here

- tf.lookup.StaticVocabularyTable : String to Id table wrapper that assigns out-of-vocabulary keys to buckets. More here

- If <other term> -> bucket_id, where bucket_id will be between 3 and 3 + num_oov_buckets - 1, calculated by: hash(<term>) % num_oov_buckets + vocab_size

- table.lookup : Looks up keys in the table, outputs the corresponding values.

In [19]:
# Create a tensor words containing words of truncated_vocabulary.
words = tf.constant(truncated_vocabulary)

In [20]:
# reate the word_ids using the corresponding indices of words in truncated_vocabulry.
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)

In [21]:
# Create the table initializer vocab_init using tf.lookup.KeyValueTensorInitializer, given the keys(here words) and the values(here word_ids) tensors.
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)

In [23]:
# Set num_oov_buckets = 1000 and create the lookup table table using tf.lookup.StaticVocabularyTable. Observe, we pass the vocab_init, num_oov_buckets as input arguments to this.
num_oov_buckets = 1000
table = tf.lookup.StaticHashTable(vocab_init, num_oov_buckets)

In [24]:
# Let's use the above table to look up the IDs of a few words:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[  22,   12,   11, 1000]])>

#### Note:
The words “this,” “movie,” and “was” were found in the table, so their IDs are lower than 10,000, while the word “faaaaaantastic” was not found, so it was mapped to one of the oov buckets, with an ID greater than or equal to 10,000.

### Creating the Final Train and Test sets
Now we will create the final training and test sets.

For creating the final training set train_set,

- we batch the reviews

- then we convert them to short sequences of words using the preprocess() function

- then encode these words using a simple encode_words() function that uses the table we just built and finally prefetch the next batch.

Let us test the model(after training) on 1000 samples of the test data as it takes a lot of time to test on the whole test set. So we shall create the final test set on 1000 samples as follows.

For creating the final test set test_set,

- we create a batch of 1000 test samples

- then we convert them to short sequences of words using the preprocess() function

- then encode these words using a simple encode_words() function that uses the table we just built.

#### Note:

- dataset.repeat().batch(32) repeatedly creates the batches of 32 samples in the dataset.

- dataset.repeat().batch(32).map(preprocess) applies the function preprocess on every batch.

- dataset.map(encode_words).prefetch(1) applies the function encode_words to the data samples and paralelly fetches the next batch.

In [25]:
# Define the encode_words() function to encode the words of train data using the lookup table table.
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

In [31]:
# Apply the function preprocess on every batch of data with 32 samples repeatedly on the train data datasets["train"].
train_set = datasets["train"].repeat().batch(32).map(preprocess)

In [32]:
# Apply the function encode_words to the train_set and parallelly fetch the next batch.
train_set = train_set.map(encode_words).prefetch(1)

In [33]:
# Similarly, apply the function preprocess on the test data datasets["test"].

test_set = datasets["test"].batch(1000).map(preprocess)

In [34]:
# Apply the function encode_words to the test_set.
test_set = test_set.map(encode_words)

In [35]:
# Let us see how the first data sample of the thus obtained train_set looks like:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


## Building the Model
- Now that we have preprocessed and created the dataset, we can create the model:

  - The first layer is an Embedding layer, which will convert word IDs into embeddings. The embedding matrix needs to have one row per word ID (vocab_size + num_oov_buckets) and one column per embedding dimension (this example uses 128 dimensions, but this is a hyperparameter you could tune).
  - Whereas the inputs of the model will be 2D tensors of shape [batch size, time steps], the output of the Embedding layer will be a 3D tensor of shape [batch size, time steps, embedding size].
#### Note:

- keras.layers.Embedding : Turns positive integers (indexes) into dense vectors of fixed size. More here.
- keras.layers.GRU : The GRU(Gated Recurrent Unit) Layer.

In [36]:
# Set embed_size to 128, which is the embedding size of each word.
embed_size = 128

- Create the model model with

  - Embedding layer

  - GRU layer with 4 units

  - GRU layer with 2 units

  - Dense layer with 1 unit and sigmoid activation

In [37]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
               mask_zero=True,
               input_shape=[None]),
    keras.layers.GRU(4, return_sequences=True),
    keras.layers.GRU(2),
    keras.layers.Dense(1, activation="sigmoid")
])

- Compile the model with "binary_crossentropy" loss(as this is a binary classification problem), "adam" optimizer and "accuracy" metric.

In [38]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## Training and Testing the Model
- It's time for training the model on the train data.

- Let us also measure the time of training using time module.

- Finally, let us test the model performance on the test data.

In [40]:
# Import time.
import time

In [52]:
# Store the start time using time.time() in start.
start = time.time()

In [53]:
# Fit the model for the train_set.
model.fit(train_set, steps_per_epoch=train_size // 32, epochs=2)

Epoch 1/2
781/781 [==============================] - 22s 28ms/step - loss: 0.2768 - accuracy: 0.8990
Epoch 2/2
781/781 [==============================] - 22s 28ms/step - loss: 0.2005 - accuracy: 0.9339


In [54]:
# Store the end time using time.time() in end.
end = time.time()

In [55]:
# Print the time of execution using (end-start).
print("Time of execution:", end-start)

Time of execution: 54.906930923461914


In [49]:
# Use model.evaluate to test the model performance on the test_set.
model.evaluate(test_set)

25/25 [==============================] - 2s 88ms/step - loss: 0.5126 - accuracy: 0.7612


[0.5125555992126465, 0.7612400054931641]

# Thank you